Crear columnas categóricas con apply()

En la lección anterior, aprendiste a crear nuevas columnas numéricas a partir de cálculos realizados en otras columnas numéricas de los datos. En esta lección, aprenderás a crear nuevas columnas categóricas que resuman datos numéricos en otras columnas. A menudo esta técnica puede simplificar el análisis y facilitar que otras personas entiendan los resultados que obtienes.

Vamos a echar otro vistazo a la columna 'year_of_release' (año de lanzamiento) en el conjunto de datos de videojuegos. En especial, queremos conocer el rango de años que cubren nuestros datos.

In [ ]:
import pandas as pd

df = pd.read_csv('/datasets/vg_sales.csv')

print(df['year_of_release'].min(), df['year_of_release'].max())

#1980.0 2020.0

¡Vaya, 40 años de juegos! Echemos un vistazo de cerca a estos valores contando cuántos juegos tenemos por año.

In [ ]:
import pandas as pd

df = pd.read_csv('/datasets/vg_sales.csv')
df_year_of_release = df['year_of_release'].value_counts()

print(df_year_of_release)

"""2008.0    1427
2009.0    1426
2010.0    1255
2007.0    1197
2011.0    1136
2006.0    1006
2005.0     939
2002.0     829
2003.0     775
2004.0     762
2012.0     653
2015.0     606
2014.0     581
2013.0     544
2016.0     502
2001.0     482
1998.0     379
2000.0     350
1999.0     338
1997.0     289
1996.0     263
1995.0     219
1994.0     121
1993.0      60
1981.0      46
1992.0      43
1991.0      41
1982.0      36
1986.0      21
1983.0      17
1989.0      17
1990.0      16
1987.0      16
1988.0      15
1984.0      14
1985.0      14
1980.0       9
2017.0       3
2020.0       1
Name: year_of_release, dtype: int64"""

Hemos recibido la respuesta, pero no está correctamente ordenada. Tenemos que ordenarla. El método value_counts() devuelve un recuento de los años, donde los años son el índice del nuevo DataFrame y los recuentos son los valores. Por lo tanto, ordenaremos nuestro resultado por los valores del índice para mostrarlo en orden cronológico.

Para ello, utilizaremos el método sort_index(). Funciona igual que el viejo sort_values(), pero se aplica a los índices en lugar de a los valores:

In [ ]:
import pandas as pd

df = pd.read_csv('/datasets/vg_sales.csv')
df_year_of_release = df['year_of_release'].value_counts().sort_index()

print(df_year_of_release)

"""
1980.0       9
1981.0      46
1982.0      36
1983.0      17
1984.0      14
1985.0      14
1986.0      21
1987.0      16
1988.0      15
1989.0      17
1990.0      16
1991.0      41
1992.0      43
1993.0      60
1994.0     121
1995.0     219
1996.0     263
1997.0     289
1998.0     379
1999.0     338
2000.0     350
2001.0     482
2002.0     829
2003.0     775
2004.0     762
2005.0     939
2006.0    1006
2007.0    1197
2008.0    1427
2009.0    1426
2010.0    1255
2011.0    1136
2012.0     653
2013.0     544
2014.0     581
2015.0     606
2016.0     502
2017.0       3
2020.0       1
Name: year_of_release, dtype: int64"""

¿Qué conclusiones podemos sacar a partir de estos datos? ¿Sería el análisis de los juegos de 2008 muy diferente del análisis de los juegos de 2009? Si queremos comparar los datos del 2000 con los de otros años, ¿debemos compararlos por separado con los de 1986 y 1987, o bien agruparlos en dos categorías: datos anteriores y posteriores a 2000?

Es difícil producir conclusiones estadísticamente relevantes para juegos de cada año cuando los juegos del conjunto de datos están tan ampliamente distribuidos en todos los años. Podemos abordar este problema al combinar nuestros clientes en categorías de grupos de edad, de modo que cada grupo sea lo suficientemente grande para que saquemos conclusiones significativas.

Categorización

Necesitamos realizar la categorización, agrupando los datos en nuevas categorías que creamos. En este caso, vamos a agrupar los juegos en cuatro categorías en función de la época.

- Los lanzados antes del año 2000 irán en la categoría 'retro'.
- Los lanzados entre el 2000 y el 2009 irán en 'modern' (moderno).
- Los lanzados a partir del 2010 irán en 'recent' (reciente).
- Los que no tienen año de lanzamiento irán en 'unknown' (desconocido).
- Queremos colocar cada juego en una de estas cuatro categorías y almacenar el resultado en una nueva columna.

Para esto no hay ninguna función prefabricada de pandas. Por fortuna, podemos escribir nuestra propia función hecha a la medida de nuestras necesidades. La función debe aceptar el año de lanzamiento como input y devolver como resultado la categoría de época para ese año.

Así se verá nuestra función personalizada era_group():

In [ ]:
def era_group(year):
    """
    La función devuelve el grupo de época de los juegos de acuerdo con el año de lanzamiento usando estas reglas:
    —'retro'   para año < 2000
    —'modern'  para 2000 <= año < 2010
    —'recent'  para año >= 2010
    —'unknown' para buscar valores año (NaN)
    """

    if year < 2000:
        return 'retro'
    elif year < 2010:
        return 'modern'
    elif year >= 2010:
        return 'recent'
    else:
        return 'unknown'

Ahora necesitaremos probar la función para cada regla. Vamos a comprobar en cuáles categorías estarían tres juegos de diferentes años:

In [ ]:
print(era_group(1983))
print(era_group(2009))
print(era_group(2021))
print(era_group(np.nan))

"""
retro
modern
recent
unknown"""

¡Excelente! La función trabaja justo como esperábamos. A continuación, crearemos una nueva columna para registrar las categorías de época.

El método apply()

Para crear una columna de categoría de época usando nuestra función personalizada era_group(), necesitaremos llamar al método apply(), el cual toma valores de una columna DataFrame y les aplica una función.

En este caso, el método apply() se debe aplicar a la columna 'year_of_release' column, porque 'year_of_release' contiene los datos que la función usará como input. La función era_group() se convierte entonces en el argumento que pasamos al método apply().

In [ ]:
import pandas as pd

def era_group(year):
    """
    La función devuelve el grupo de época de los juegos de acuerdo con el año de lanzamiento usando estas reglas:
    —'retro'   para año < 2000
    —'modern'  para 2000 <= año < 2010
    —'recent'  para año >= 2010
    —'unknown' para buscar valores año (NaN)
    """

    if year < 2000:
        return 'retro'
    elif year < 2010:
        return 'modern'
    elif year >= 2010:
        return 'recent'
    else:
        return 'unknown'


df = pd.read_csv('/datasets/vg_sales.csv')

df['era_group'] = df['year_of_release'].apply(era_group)
print(df.head())

"""              name platform  year_of_release         genre publisher  \
0                Wii Sports      Wii           2006.0        Sports  Nintendo   
1         Super Mario Bros.      NES           1985.0      Platform  Nintendo   
2            Mario Kart Wii      Wii           2008.0        Racing  Nintendo   
3         Wii Sports Resort      Wii           2009.0        Sports  Nintendo   
4  Pokemon Red/Pokemon Blue       GB           1996.0  Role-Playing  Nintendo   

  developer  na_sales  eu_sales  jp_sales  critic_score  user_score era_group  
0  Nintendo     41.36     28.96      3.77          76.0         8.0    modern  
1       NaN     29.08      3.58      6.81           NaN         NaN     retro  
2  Nintendo     15.68     12.76      3.79          82.0         8.3    modern  
3  Nintendo     15.61     10.93      3.28          80.0         8.0    modern  
4       NaN     11.27      8.89     10.22           NaN         NaN     retro"""

Así de sencillo es crear una columna con nuestra función personalizada: ¡tan solo una línea de código!

Ahora vamos a analizar los datos en los grupos de época con el método value_counts():

In [ ]:
import pandas as pd

def era_group(year):
    """
    La función devuelve el grupo de época de los juegos de acuerdo con el año de lanzamiento usando estas reglas:
    —'retro'   para año < 2000
    —'modern'  para 2000 <= año < 2010
    —'recent'  para año >= 2010
    —'unknown' para buscar valores año (NaN)
    """

    if year < 2000:
        return 'retro'
    elif year < 2010:
        return 'modern'
    elif year >= 2010:
        return 'recent'
    else:
        return 'unknown'


df = pd.read_csv('/datasets/vg_sales.csv')

df['era_group'] = df['year_of_release'].apply(era_group)
print(df['era_group'].value_counts())

"""
modern     9193
recent     5281
retro      1974
unknown     269
Name: era_group, dtype: int64"""

Los datos están listos para el análisis.

Ahora te toca a ti. En los ejercicios, practica usar apply() para categorizar los videojuegos según sus calificaciones y analiza cómo se relaciona eso con las ventas.

1.

Comienza por escribir una función llamada score_group() que organice los juegos por categorías de acuerdo con las puntuaciones de las críticas. Categoriza las puntuaciones con base en estas características:

valor 'low' (bajo) para puntuaciones menores a 60.
valor 'medium' (medio) para puntuaciones de 60 a 79.
valor 'high' (alto) para puntuaciones mayores a 80.
valor 'no score' (sin puntuación) para puntuaciones sin valores.
La función score_group() debe tener un input numérico llamado score. La salida debe ser una cadena que designe la categoría de la puntuación.

Asegúrate de que tu función produzca el output correcto cuando se le pasen los valores 10, 65, 99 y np.nan. Escribimos una declaración print() distinta para llamar a cada función.

No dudes en copiar nuestra función del ejemplo anterior y adaptarla a tus necesidades actuales.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('/datasets/vg_sales.csv')

def score_group(score):
    '''La funcion devuelve los juegos por categoria de acuerdo con las puntuaciones de las criticas'''
    
    if score < 60:
        return 'low'
    elif score < 80:
        return 'medium'
    elif score >= 80:
        return 'high'
    else:
        return 'no score'
        # escribe aquí la definición de tu función

# imprime los resultados de llamar a la función con estos inputs en orden: 10, 65, 99, np.nan
print(score_group(10))
print(score_group(65))
print(score_group(99))
print(score_group(np.nan))

"""
low
medium
high
no score"""

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/vg_sales.csv'